# AI-Based Real-Time Face Monitoring System for Online Proctoring 
## (Prevent Proxy / Impersonation in Online Exams & Interviews)

---

## 1. Business Understanding

### 1.1 Business Problem

Online exams, interviews, and certification tests have become widespread across:

- Universities and colleges  
- Recruitment platforms  
- Online certification providers  
- Corporate assessments  

However, remote assessments face serious integrity challenges, such as:

- Candidates leaving their seats during exams  
- Proxy candidates replacing the original person  
- Multiple people appearing on camera  
- Lack of continuous human monitoring  

These issues lead to:

- Compromised exam credibility  
- Unfair evaluation  
- Loss of trust in online assessment systems  

From an organizational and business perspective, impersonation results in:

- Invalid assessment outcomes  
- Increased manual proctoring costs  
- Legal and compliance risks  
- Brand reputation damage  
- Reduced acceptance of online examinations  

Manual monitoring using human proctors is:

- Expensive  
- Error-prone  
- Not scalable for large-scale exams  

---

### 1.2 Business Objective

The primary objective of the **AI-Based Real-Time Face Monitoring System for Online Proctoring** is to:

- Continuously monitor the candidate through webcam video  
- Detect face absence during an active session  
- Detect multiple faces indicating possible impersonation  
- Generate real-time alerts and logs for suspicious activity  
- Improve fairness and integrity in online examinations  

Secondary objectives include:

- Reducing dependency on manual invigilators  
- Providing automated evidence for exam violations  
- Enabling scalable remote assessment monitoring  
- Supporting cost-effective proctoring solutions  

---

### 1.3 Business Constraints

| Constraint Type | Description |
|----------------|-------------|
| Cost Constraint | The system should work using a standard webcam without requiring expensive sensors or hardware. |
| Latency Constraint | Face detection and alerts must occur in real time to prevent misuse during the exam. |
| Environmental Constraint | Performance may vary with lighting conditions, camera quality, and background environment. |
| Privacy Constraint | Captured video and images must comply with data privacy, consent, and institutional policies. |
| Accuracy Constraint | False warnings should be minimized to avoid unnecessary interruptions. |
| Hardware Constraint | The system should run efficiently on low-end devices such as laptops and basic PCs. |

---

## 5. Future Goals & Roadmap

### 5.1 Face Recognition Enhancement
- Integrate facial recognition for candidate identity verification  
- Detect impersonation even when only one face is present  

### 5.2 Behavioral Monitoring
- Head movement analysis  
- Eye gaze and attention tracking  
- Long-term face absence detection  

### 5.3 Multi-Source Monitoring
- Screen activity monitoring  
- Multiple camera support for high-security exams  

### 5.4 Cloud-Based Exam Analytics
- Centralized dashboards for:
  - Violation statistics  
  - Candidate risk analysis  
  - Exam integrity reports  

### 5.5 Integration with Examination Platforms
- Learning Management Systems (LMS)  
- Online exam portals  
- Automated compliance and audit reports  

---

## 6. Future Market Opportunity

The rapid growth of online education and remote hiring has increased demand for:

- Automated online proctoring solutions  
- AI-assisted exam monitoring systems  
- Secure digital assessment platforms  

High adoption potential exists in:

- Universities and educational institutions  
- Corporate recruitment and assessments  
- Government and competitive examinations  
- Certification bodies  

Emerging markets such as **India, Southeast Asia, and the Middle East** show strong growth in online assessment adoption.

---

## 7. Summary (Executive View)

The **AI-Based Real-Time Face Monitoring System for Online Proctoring** is a practical and cost-effective solution that:

- Enhances integrity of online exams  
- Prevents proxy and impersonation attempts  
- Reduces operational and monitoring costs  
- Improves trust in remote assessments  
- Supports scalable digital examination systems  

This system serves as a foundational component for:

- Intelligent online proctoring platforms  
- Secure remote assessments  
- AI-enabled education technologies  


In [7]:
import cv2
import mediapipe as mp
import datetime
import csv
import os
import winsound
import time

# -------------------- SETUP --------------------

os.makedirs("screenshots", exist_ok=True)

csv_exists = os.path.exists("violation_log.csv")
csv_file = open("violation_log.csv", "a", newline="")
csv_writer = csv.writer(csv_file)
if not csv_exists:
    csv_writer.writerow(["Time", "Event", "Image"])

mp_face_detection = mp.solutions.face_detection
face_detection = mp_face_detection.FaceDetection(
    min_detection_confidence=0.7,
    model_selection=0
)
drawing = mp.solutions.drawing_utils

video = cv2.VideoCapture(0)

# -------------------- STATE VARIABLES --------------------

baseline_x = None
MOVE_THRESHOLD = 0.04

previous_direction = "CENTER"
direction_start_time = None
DIRECTION_HOLD_TIME = 1.0

last_event_time = 0
COOLDOWN = 2

left_count = 0
right_count = 0

# -------------------- MAIN LOOP --------------------

while video.isOpened():
    ret, frame = video.read()
    if not ret:
        break

    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_detection.process(rgb)

    face_count = len(results.detections) if results.detections else 0
    direction = "CENTER"
    status = "OK"
    violation = False

    # -------- FACE STATUS --------
    if face_count == 0:
        status = "FACE MISSING"
        violation = True
        baseline_x = None
    elif face_count > 1:
        status = "MULTIPLE FACES"
        violation = True

    # -------- LEFT / RIGHT DETECTION --------
    if results.detections and face_count == 1:
        det = results.detections[0]
        drawing.draw_detection(frame, det)

        bbox = det.location_data.relative_bounding_box
        center_x = bbox.xmin + bbox.width / 2

        if baseline_x is None:
            baseline_x = center_x

        diff = center_x - baseline_x
        if diff < -MOVE_THRESHOLD:
            direction = "LEFT"
        elif diff > MOVE_THRESHOLD:
            direction = "RIGHT"

    now = time.time()

    # -------- DIRECTION HOLD LOGIC --------
    if direction != previous_direction:
        direction_start_time = now
        previous_direction = direction

    elif direction in ["LEFT", "RIGHT"] and direction_start_time:
        if now - direction_start_time >= DIRECTION_HOLD_TIME:
            if direction == "LEFT":
                left_count += 1
            else:
                right_count += 1
            violation = True
            direction_start_time = None

    # -------- LOGGING --------
    if violation and (now - last_event_time) > COOLDOWN:
        winsound.Beep(2000, 300)

        ts = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
        event = status if status != "OK" else direction.lower()
        img_path = f"screenshots/{event}_{ts}.jpg"

        cv2.imwrite(img_path, frame)
        csv_writer.writerow([ts, event, img_path])

        last_event_time = now

    # -------------------- UI (PROFESSIONAL HUD) --------------------

    # Overlay panel
    overlay = frame.copy()
    cv2.rectangle(overlay, (10, 10), (330, 140), (30, 30, 30), -1)
    cv2.addWeighted(overlay, 0.7, frame, 0.3, 0, frame)
    cv2.rectangle(frame, (10, 10), (330, 140), (200, 200, 200), 1)

    # Text info
    cv2.putText(frame, f"Faces : {face_count}", (20, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2)
    cv2.putText(frame, f"Left  : {left_count}", (20, 70),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2)
    cv2.putText(frame, f"Right : {right_count}", (20, 100),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2)

    # -------- TRAFFIC LIGHT (FIXED) --------
    light_color = (0,255,0) if not violation else (0,0,255)
    cv2.circle(frame, (300, 45), 10, light_color, -1)

    # Status text
    cv2.putText(frame, f"STATUS: {status}", (20, 130),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, light_color, 2)

    cv2.imshow("AI-Based Face Monitoring System", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# -------------------- CLEANUP --------------------

video.release()
csv_file.close()
cv2.destroyAllWindows()
